In [1]:
#This code contains the codes to identify regulons using SCENIC. 
#It required a processed gene expression matrix (processed_data.csv), cluster information (cluster.csv), 
#different reductions (reductions.csv: example umap.csv, pca.csv), and metadata (meta.csv) as input. The same code can be used for both 
#Cortex and CB.

2023-03-19 05:53:45.733585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 05:53:45.850625: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 05:53:45.887112: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-19 05:53:47.077807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:
import os
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE

import pickle

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns

import json
import zlib
import base64
import umap

In [2]:
f_loom_path_scenic = "./loom/filtered_scenic.loom"
f_pyscenic_output = "./loom/pyscenic_output.loom"

adata = sc.read_csv("./scenic/processed_data.csv")
adata = adata.transpose()

In [3]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)


In [4]:
f_tfs = "./resources/mm_mgi_tfs.txt"
!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 20



2023-03-19 05:54:00,266 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2023-03-19 05:54:00,827 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
20 partitions
computing dask graph
not shutting down client, client was created externally
finished

2023-03-19 05:58:28,752 - pyscenic.cli.pyscenic - INFO - Writing results to file.


In [5]:
adjacencies = pd.read_csv("adj.csv", index_col=False, sep=',')
adjacencies.head()

,TF,target,importance
0,Snrnp70,Son,124.281202
1,Luzp2,Slc1a3,123.981223
2,Snrnp70,Dbi,110.965503
3,Luzp2,Sparcl1,107.915980
4,Prnp,Cd81,105.846450


In [6]:
import glob
# ranking databases
f_db_glob = "./databases/notUsing/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "./resources/motifs-v9-nr.mgi-m0.001-o0.0.tbl"

!pyscenic ctx adj.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20



2023-03-19 05:58:40,901 - pyscenic.cli.pyscenic - INFO - Creating modules.

2023-03-19 05:58:41,413 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2023-03-19 05:58:41,934 - pyscenic.utils - INFO - Calculating Pearson correlations.

2023-03-19 05:58:42,163 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2023-03-19 05:58:45,845 - pyscenic.utils - INFO - Creating modules.

2023-03-19 05:59:43,778 - pyscenic.cli.pyscenic - IN


2023-03-19 05:59:57,226 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(3): motif annotations loaded in memory.

2023-03-19 05:59:57,226 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(3): motif annotations loaded in memory.

2023-03-19 05:59:57,286 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(2): motif annotations loaded in memory.

2023-03-19 05:59:57,286 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(2): motif annotations loaded in memory.

2023-03-19 05:59:57,377 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(1): motif annotations loaded in memory.

2023-03-19 05:59:57,377 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(1): motif annotations loaded in memory.

2023-03-19 05:59:57,451 - pyscenic.prun


2023-03-19 06:00:10,203 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zkscan5 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:00:10,218 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zkscan17 could be mapped to mm9-tss-centered-5kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:00:10,341 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zkscan8 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:00:10,363 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zkscan5 could be mapped to mm9-tss-centered-5kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:00:10,502 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zkscan8 could be mapped to mm9-tss-centered-5kb-7spec


2023-03-19 06:02:48,311 - pyscenic.transform - WARNING - Less than 80% of the genes in Gtf3a could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:02:51,711 - pyscenic.transform - WARNING - Less than 80% of the genes in Gtf3a could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:02:52,102 - pyscenic.transform - WARNING - Less than 80% of the genes in Grhl1 could be mapped to mm9-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:02:56,181 - pyscenic.transform - WARNING - Less than 80% of the genes in Hltf could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:02:56,244 - pyscenic.transform - WARNING - Less than 80% of the genes in Hmbox1 could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19


2023-03-19 06:04:12,631 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Jun could be mapped to mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:04:13,255 - pyscenic.transform - WARNING - Less than 80% of the genes in Nucb1 could be mapped to mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:04:18,025 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Klf4 could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:04:18,708 - pyscenic.transform - WARNING - Less than 80% of the genes in Phf21a could be mapped to mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:04:20,717 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Klf4 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skippin

...


2023-03-19 06:15:01,638 - pyscenic.transform - WARNING - Less than 80% of the genes in Supt20 could be mapped to mm9-tss-centered-5kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:02,192 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp692 could be mapped to mm9-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:02,310 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp706 could be mapped to mm9-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:03,176 - pyscenic.transform - WARNING - Less than 80% of the genes in Taf1 could be mapped to mm9-tss-centered-5kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:03,437 - pyscenic.transform - WARNING - Less than 80% of the genes in Zscan29 could be mapped to mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:


2023-03-19 06:15:24,241 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zfp426 could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:25,431 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zfp560 could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:26,607 - pyscenic.transform - WARNING - Less than 80% of the genes in Supt20 could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:27,783 - pyscenic.transform - WARNING - Less than 80% of the genes in Ubp1 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:27,819 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Zbtb6 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.ran


2023-03-19 06:15:56,528 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp706 could be mapped to mm9-tss-centered-5kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:57,072 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp787 could be mapped to mm9-tss-centered-5kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:15:57,463 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp574 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:02,501 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp532 could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:02,588 - pyscenic.prune - INFO - Worker mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings(1): All regulons derived.

2023-03-19 06:16:02,588 - pyscenic.prune - INFO - Worker mm


2023-03-19 06:16:13,410 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp976 could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:13,467 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfx could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:13,957 - pyscenic.transform - WARNING - Less than 80% of the genes in Zkscan17 could be mapped to mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:14,999 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp971 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:15,052 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfp974 could be mapped to mm9-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-


2023-03-19 06:16:42,287 - pyscenic.transform - WARNING - Less than 80% of the genes in Zfx could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:42,772 - pyscenic.transform - WARNING - Less than 80% of the genes in Zkscan17 could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:43,549 - pyscenic.transform - WARNING - Less than 80% of the genes in Zscan29 could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:43,867 - pyscenic.transform - WARNING - Less than 80% of the genes in 2010315B03Rik could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.

2023-03-19 06:16:44,243 - pyscenic.transform - WARNING - Less than 80% of the genes in AU041133 could be mapped to mm9-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this

In [7]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output}


2023-03-19 06:17:07,534 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2023-03-19 06:17:08,010 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2023-03-19 06:17:21,447 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2023-03-19 06:17:34,133 - pyscenic.cli.pyscenic - INFO - Writing results to file.
/home/sandesh.acharya/software/miniconda3/envs/notebook/lib/python3.9/site-packages/pyscenic/cli/utils.py:347: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, threshold in auc_thresholds.iteritems()


In [9]:
# collect SCENIC AUCell output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [10]:
# UMAP
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric='correlation').fit_transform
dr_umap = runUmap( auc_mtx )
pd.DataFrame(dr_umap, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_umap.txt", sep='\t')
# tSNE
tsne = TSNE( n_jobs=20 )
dr_tsne = tsne.fit_transform( auc_mtx )
pd.DataFrame(dr_tsne, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_tsne.txt", sep='\t')

# scenic output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))
#exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID)
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
regulons = lf.ra.Regulons
dr_umap = pd.read_csv( 'scenic_umap.txt', sep='\t', header=0, index_col=0 )
dr_tsne = pd.read_csv( 'scenic_tsne.txt', sep='\t', header=0, index_col=0 )
###


In [11]:
auc_mtx.columns = auc_mtx.columns.str.replace('\(','_(')
regulons.dtype.names = tuple( [ x.replace("(","_(") for x in regulons.dtype.names ] )
# regulon thresholds
rt = meta['regulonThresholds']
for i,x in enumerate(rt):
    tmp = x.get('regulon').replace("(","_(")
    x.update( {'regulon': tmp} )

/tmp/ipykernel_1421503/1306291957.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  auc_mtx.columns = auc_mtx.columns.str.replace('\(','_(')


In [12]:
##adata include the umap and pca embeddings
tsne = TSNE( n_jobs=20 )
adata.obsm['X_tsne'] = tsne.fit_transform( adata.X )

harmony = pd.read_csv("./reductions/harmony_embeddings.csv", header = 0, index_col = 0)
pca = pd.read_csv("./reductions/pca_embeddings.csv", header = 0, index_col = 0)
umap = pd.read_csv("./reductions/umap_embeddings.csv", header = 0, index_col = 0)
pcaa = pca.iloc[:, :2].values
harr = harmony.iloc[:, :2].values
umapp = umap.iloc[:, :2].values
adata.obsm['X_pca'] = pcaa
adata.obsm['X_umap'] = umapp
#adata.obsm['X_harmony'] = harmony

In [13]:
tsneDF = pd.DataFrame(adata.obsm['X_tsne'], columns=['_X', '_Y'])

Embeddings_X = pd.DataFrame( index=lf.ca.CellID )

Embeddings_X = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[0] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[0] ,
        dr_tsne['X'] ,
        dr_umap['X']
    ], sort=False, axis=1, join='outer' )
Embeddings_X.columns = ['1','2','3','4']

Embeddings_Y = pd.DataFrame( index=lf.ca.CellID )
Embeddings_Y = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[1] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[1] ,
        dr_tsne['Y'] ,
        dr_umap['Y']
    ], sort=False, axis=1, join='outer' )
Embeddings_Y.columns = ['1','2','3','4']

In [14]:
cluster_info = pd.read_csv("./scenic/clusters.csv", header = 0, index_col = 0)
cluster_info.index = cluster_info.index.str.replace("-", ".")
sum(adata.obs.index == cluster_info.index)
adata.obs['cluster_ids'] = cluster_info['Idents.processed.']

In [15]:
annot = pd.read_csv("./scenic/meta.csv", header = 0, index_col = 0)
annot.index = annot.index.str.replace("-", ".")
print(sum(adata.obs.index == annot.index))
adata.obs['mutation'] = annot['group.name']

2895


In [16]:
### metadata
metaJson = {}

metaJson['embeddings'] = [
    {
        "id": -1,
        "name": f"Scanpy tSNE"
    },
    {
        "id": 1,
        "name": f"Seurat UMAP"
    },
    {
        "id": 2,
        "name": "Seurat PCA"
    },
    {
        "id": 3,
        "name": "SCENIC AUC tSNE"
    },
    {
        "id": 4,
        "name": "SCENIC AUC UMAP"
    },
]

metaJson["clusterings"] = [{
            "id": 0,
            "group": "Scanpy",
            "name": "Scanpy louvain default resolution",
            "clusters": [],
        }]

metaJson["metrics"] = [
        {
            "name": "nUMI"
        }, {
            "name": "nGene"
        }, {
            "name": "Percent_mito"
        }
]

metaJson["annotations"] = [
    {
        "name": "Louvain_clusters_Seurat",
        "values": list(set( adata.obs['cluster_ids'].astype(np.str) ))
    },
    {
        "name": "Sample",
        "values": list(set(adata.obs['mutation'].values))
    }
]

# SCENIC regulon thresholds:
metaJson["regulonThresholds"] = rt

/tmp/ipykernel_1421503/1704344211.py:47: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  "values": list(set( adata.obs['cluster_ids'].astype(np.str) ))


In [17]:
for i in range(max(set([int(x) for x in adata.obs['cluster_ids']])) + 1):
    clustDict = {}
    clustDict['id'] = i
    clustDict['description'] = f'Unannotated Cluster {i + 1}'
    metaJson['clusterings'][0]['clusters'].append(clustDict)
    
clusterings = pd.DataFrame()
clusterings["0"] = adata.obs['cluster_ids'].values.astype(np.int64)

In [18]:
sum(clusterings['0'] == cluster_info['Idents.processed.'].values)

2895

In [19]:
def dfToNamedMatrix(df):
    arr_ip = [tuple(i) for i in df.values]
    dtyp = np.dtype(list(zip(df.dtypes.index, df.dtypes)))
    arr = np.array(arr_ip, dtype=dtyp)
    return arr

col_attrs = {
    "CellID": np.array(adata.obs.index),
    "nUMI": np.array(np.array(np.sum(adata.X.transpose() , axis=0)).flatten()),
    "nGene": np.array(np.sum(adata.X.transpose()>0 , axis=0)).flatten(),
    "Louvain_clusters_Scanpy": np.array( adata.obs['cluster_ids'].values),
    "Embedding": dfToNamedMatrix(tsneDF),
    "Embeddings_X": dfToNamedMatrix(Embeddings_X),
    "Embeddings_Y": dfToNamedMatrix(Embeddings_Y),
    "RegulonsAUC": dfToNamedMatrix(auc_mtx),
    "Clusterings": dfToNamedMatrix(clusterings),
    'samples': np.array(adata.obs['mutation'].values),
    "ClusterID": np.array(adata.obs['cluster_ids'].values)
}

row_attrs = {
    "Gene": lf.ra.Gene,
    "Regulons": regulons,
}

attrs = {
    "title": "sampleTitle",
    "MetaData": json.dumps(metaJson),
    "Genome": 'hg38',
    "SCopeTreeL1": "",
    "SCopeTreeL2": "",
    "SCopeTreeL3": ""
}

# compress the metadata field:
attrs['MetaData'] = base64.b64encode(zlib.compress(json.dumps(metaJson).encode('ascii'))).decode('ascii')

f_final_loom = './loom/scenic_integrated-output.loom'

lp.create(
    filename = f_final_loom ,
    layers=lf[:,:],
    row_attrs=row_attrs, 
    col_attrs=col_attrs, 
    file_attrs=attrs
)
lf.close()